In [7]:
import numpy as np
import warnings
from joblib import Parallel, delayed
from numba import jit

In [8]:
def user_pipeline(data):
    return np.mean(data, axis=1, keepdims=True)

@jit(nopython=True)
def loo_(data):
    n_features, n_samples = data.shape
    loo_means = np.zeros((n_features, n_samples))
    loo_stds = np.zeros((n_features, n_samples))

    for i in range(n_features):
        v = data[i]
        s, s2 = np.sum(v), np.sum(v**2)
        means = (s - v) / (n_samples - 1)
        vars_ = ((s2 - v**2) - (n_samples - 1) * means**2) / (n_samples - 2)
        loo_means[i], loo_stds[i] = means, np.sqrt(vars_)

    return loo_means, loo_stds

@jit(nopython=True)
def gen_noise(chol_factor, sensitivity, max_attempts=10000):
    dim = sensitivity.shape[0]
    for _ in range(max_attempts):
        z = np.random.standard_normal(dim)
        noise = chol_factor @ z
        if np.all(noise >= sensitivity):
            return noise, True
    return noise, False

def dp(data, original_output, epsilon=1.0):
    loo_means, loo_stds = loo_(data)
    sensitivity = np.max(np.abs(loo_means - original_output), axis=1)

    loo_scales = np.std(loo_stds, axis=1) / epsilon
    cov = np.cov(data)
    scale_factors = 2 * loo_scales / np.sqrt(np.diag(cov))
    scale_matrix = np.diag(scale_factors)

    noise_cov = scale_matrix @ cov @ scale_matrix
    if np.min(np.linalg.eigvals(noise_cov)) <= 1e-10:
        noise_cov += np.eye(data.shape[0]) * 1e-8

    try:
        chol = np.linalg.cholesky(noise_cov)
        noise, success = gen_noise(chol, sensitivity)
        if not success:
            raise ValueError("Correlated noise generation failed")
    except (np.linalg.LinAlgError, ValueError):
        warnings.warn("Failed to generate correlated noise, using independent noise")
        noise = np.array([np.random.normal(0, 2 * s) for s in loo_scales])
        for i in range(len(noise)):
            while abs(noise[i]) < sensitivity[i]:
                noise[i] = np.random.normal(0, 2 * loo_scales[i])

    noisy_outputs = original_output.flatten() + noise
    return noisy_outputs, sensitivity

In [9]:
size = [300,50000]
test = np.random.normal(loc=0.0, scale=1.0, size=size)

In [17]:
def main(data):
    original_output = user_pipeline(data)
    with warnings.catch_warnings(record=True) as w:
        warnings.simplefilter("always")
        noisy_outputs, sensitivities = dp(data, original_output)
        failed = any("Failed to generate correlated noise" in str(msg.message) for msg in w)

    result = {f'dp_t{i+1}': noisy_outputs[i] for i in range(len(noisy_outputs))}
    result.update({f'sensitivity_t{i+1}': sensitivities[i] for i in range(len(sensitivities))})
    return result, int(failed)


if __name__ == "__main__":
    #cov_matrix = np.array([[1.0, 0.5], [0.5, 1.0]])
    #data = np.random.multivariate_normal([0, 0], cov_matrix, 20000).T
    
    cov_matrix = np.array([[1.0, 0.5, 0.3],
                           [0.5, 1.0, 0.2],
                           [0.3, 0.2, 1.0]])
    data = np.random.multivariate_normal([0, 0, 0], cov_matrix, 20000).T

    results = Parallel(n_jobs=20, verbose=1, batch_size=1)(
        delayed(main)(data) for _ in range(2000)
    )


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    1.3s
[Parallel(n_jobs=20)]: Done 760 tasks      | elapsed:    1.9s
[Parallel(n_jobs=20)]: Done 1210 tasks      | elapsed:    2.8s
[Parallel(n_jobs=20)]: Done 1760 tasks      | elapsed:    3.9s
[Parallel(n_jobs=20)]: Done 2000 out of 2000 | elapsed:    4.3s finished
